# Dataset for Aspect-Based Sentiment Analysis and Topic Modeling on Borobudur Temple and Prambanan Temple by Dian Arianto
https://github.com/dian9395/dataset-analisis-sentimen-berbasis-aspek-dan-pemodelan-topik

In [38]:
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
df = pd.read_excel('dataset/reviews_borobudur_prambanan_TripAdvisor_GMaps_all_tesis.xlsx')
df_clean = df.dropna()
# print("\nDrop rows with any NaN values:")
df_clean

,id,lokasi,text,daya_tarik,amenitas,aksesibilitas,citra,harga,sdm
0,1.0,Candi Borobudur,peninggalan sejarah yang sudah berumur 1200 ta...,1,-,-,1,0,0
1,2.0,Candi Borobudur,Pertama kali bepergian selama masa pandemi. Ca...,-,1,-,1,-,1
2,3.0,Candi Borobudur,"Candi Borobudur di Magelang, Yogyakarta adalah...",1,-,-,1,-,-
3,4.0,Candi Borobudur,"Baru pertama kali kesini, pas sih kalau tempat...",1,-,-,-,-1,-
4,5.0,Candi Borobudur,"candi borobudur, tempat wisata ini sudah terke...",1,1,-,1,-,-
...,...,...,...,...,...,...,...,...,...
6738,6739.0,Candi Prambanan,The place is great and probably everything you...,1,-,-,-,-1,-
6739,6740.0,Candi Prambanan,Prambanan or Rara Jonggrang (Javanese: ꦫꦫꦗꦺꦴꦁꦒ...,0,-,-,-,-,-
6740,6741.0,Candi Prambanan,This temple is not worth the price. There’s no...,-1,-,-,-1,-1,-1
6741,6742.0,Candi Prambanan,Prambanan temple is the largest hindu temple o...,1,-,-,-,-,-


In [29]:
review = df_clean['text']

## Preprocessing
Emoji processing; case folding; menghapus username, angka, dan tanda baca; koreksi ejaan dan singkatan serta menghapus whitespace; penghapusan stopwords, dan stemming.

In [30]:
import re
# Daftar emoji berdasarkan sentimen
emoji_positif = ["😊", "😄", "♥", "😍", "😘", "😃", "😁", "😆", "😇", "☺"]
emoji_negatif = ["😢", "😠", "😡", "😭", "😱", "😨", "😫", "😩", "😖", "😔"]
emoji_netral = ["👍", "✨", "★", "█", "👌", "♫", "�", "©", "💬", "🔔"]

# Fungsi untuk mengganti emoji dengan label sentimen
def ganti_emoji(teks):
    for emoji in emoji_positif:
        teks = teks.replace(emoji, "positif")
    for emoji in emoji_negatif:
        teks = teks.replace(emoji, "negatif")
    for emoji in emoji_netral:
        teks = teks.replace(emoji, "netral")
    return teks

teks = review.apply(ganti_emoji)

In [13]:
import gdown
# URL dari file Google Drive
stopwords_uci_link = 'https://drive.google.com/uc?id=1qqSuE1tuDCCfsdKgXMC3Nj0jda1LuWXB'
stopwords_nltk_link = 'https://drive.google.com/uc?id=1Ufurgc02rF2_yuFh8GNw1VOpkNbJLKWx'
stopwords_uci_output = 'stopwords_uci.txt'
stopwords_nltk_output = 'stopwords_nltk.txt'
# Mengunduh file
gdown.download(stopwords_uci_link, stopwords_uci_output, quiet=False)
gdown.download(stopwords_nltk_link, stopwords_nltk_output, quiet=False)
# Membaca konten file
with open(stopwords_uci_output, 'r', encoding='utf-8') as file:
    stopwords_uci = file.read()
with open(stopwords_nltk_output, 'r', encoding='utf-8') as file:
    stopwords_nltk = file.read()

Downloading...
From: https://drive.google.com/uc?id=1qqSuE1tuDCCfsdKgXMC3Nj0jda1LuWXB
To: e:\PROJECT\Python-Pro\sa-TripAdvisor\stopwords_uci.txt
100%|██████████| 6.91k/6.91k [00:00<00:00, 1.00MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ufurgc02rF2_yuFh8GNw1VOpkNbJLKWx
To: e:\PROJECT\Python-Pro\sa-TripAdvisor\stopwords_nltk.txt
100%|██████████| 7.39k/7.39k [00:00<00:00, 724kB/s]


In [25]:
def preprocess(text):
    # text preprocessing
    teks = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    teks = teks.lower()
    stop_words = set(stopwords.words('english') + stopwords_uci.split('\n') + stopwords_nltk.split('\n'))
    tokens = word_tokenize(teks)
    filtered_tokens = [word for word in tokens if word.isalnum() and not any(char.isdigit() for char in word) and word not in stop_words]

    stemmer_factory = StemmerFactory()
    sastrawi_stemmer = stemmer_factory.create_stemmer()
    lemmatizer = nltk.WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    indonesian_stemmed_words = [sastrawi_stemmer.stem(word) for word in lemma]
    clean_reviews = ' '.join(indonesian_stemmed_words)
    return clean_reviews

In [36]:
teks = teks.apply(preprocess)
teks

0       tinggal sejarah umur indonesia bangsa indonesi...
1       kali pergi pandemi candi borobudur protokol se...
2       candi borobudur magelang yogyakarta salah reko...
3       kali kesini pa sih ajaib dunia sulit bayang ba...
4       candi borobudur wisata kenal dunia ganti sy wi...
                              ...                        
6738    place great probably everything expect however...
6739    prambanan rara jonggrang javanese romanized ra...
6740    temple worth price there much see poorly maint...
6741    prambanan temple largest hindu temple ancient ...
6742    best choice three mostfamous temple yogyakarta...
Name: text, Length: 6743, dtype: object

## Ekstraksi Fitur
words n-gram. words unigram+bigram+trigram untuk fiturnya dan vektornya diekstraksi dengan CountVectorizer dari pustaka Scikit-Learn

In [39]:
# Menginisialisasi CountVectorizer dengan pengaturan untuk words unigram, bigram, dan trigram
vectorizer = CountVectorizer(ngram_range=(1, 3))

# Melakukan ekstraksi fitur pada data teks
X = vectorizer.fit_transform(teks)

# Mendapatkan daftar fitur (kata-kata) yang diekstraksi
feature_names = vectorizer.get_feature_names_out()

# Menampilkan hasil ekstraksi fitur
print("Daftar fitur (kata-kata) yang diekstraksi:")
print(feature_names)

# Menampilkan vektor representasi dari setiap kalimat
print("Vektor representasi dari setiap kalimat:")
print(X.toarray())

Daftar fitur (kata-kata) yang diekstraksi:
['aaaa' 'aaaa let' 'aaaa let know' ... 'zuri' 'zuri malioboro'
 'zuri malioboro langsung']
Vektor representasi dari setiap kalimat:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## Eksperimen

### Analisis Sentimen Berbasis Aspek

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

### Pemodelan Topik

## Evaluasi